This notebook is going to focus on processing the players' match statistics only. The other three files will be processed in separate notebooks.

In [1]:
# Importing standard packages for working with dataframes.
import numpy as np
import pandas as pd


# Unlike in the Stage 1 notebooks, we are going to create new variables rather than perform the operations in-place here.
# The reason is that we might need to review the original data during processing.
raw_players_match = pd.read_csv('../raw_data/raw_players_match.csv')

# Does everything seem to be alright with the data?
raw_players_match.head(5)

,URL,Player name,IDSeason,Season,Team,Date,Teams,Score,№,G,...,BLS,FOA,W,L,SOP,GA,Sv,%Sv,GAA,SO
0,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season 2013/2014,54,28 Dec 2013,Barys - Amur,8:2,91,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season 2013/2014,54,3 Jan 2014,Amur - Lokomotiv,2:1,91,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season 2013/2014,54,5 Jan 2014,Amur - SKA,1:6,91,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season 2013/2014,54,7 Jan 2014,Amur - Atlant,2:3 Б,91,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season 2013/2014,54,9 Jan 2014,Amur - Severstal,1:3,91,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can already see that there are some issues with missing data. In addition, the player's team is only indicated by an ID rather than its official name.

In [2]:
# What would the summary tell us?
raw_players_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451106 entries, 0 to 451105
Data columns (total 40 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   URL          451106 non-null  object 
 1   Player name  451106 non-null  object 
 2   IDSeason     451106 non-null  int64  
 3   Season       451106 non-null  object 
 4   Team         451106 non-null  int64  
 5   Date         451106 non-null  object 
 6   Teams        451106 non-null  object 
 7   Score        451106 non-null  object 
 8   №            451106 non-null  int64  
 9   G            451106 non-null  int64  
 10  Assists      451106 non-null  int64  
 11  PTS          409063 non-null  float64
 12  +/-          409063 non-null  float64
 13  +            409063 non-null  float64
 14  -            409063 non-null  float64
 15  PIM          451106 non-null  int64  
 16  ESG          409063 non-null  float64
 17  PPG          409063 non-null  float64
 18  SHG          409063 non-

We can see that in many columns there is no missing data at all. Some columns are stored as floats while they should in fact be integers. At the same time, for other columns there is a clear separation into skaters (forwards and defencemen) and goalies.

For example, we can see that season statistics appears to have 409063 rows of data for skaters and 42020 rows for goalies, with a total of 451083 rows. However, there are 451106 rows in the dataframe so 23 rows seem to be unaccounted in either.

Let us find out who is messing up our data. We can see that icetime has exactly 451083 non-null values which is in line with our calculations, so we are probably interested in the cases when icetime is null.

In [3]:
raw_players_match[raw_players_match['TOI'].isnull()]

,URL,Player name,IDSeason,Season,Team,Date,Teams,Score,№,G,...,BLS,FOA,W,L,SOP,GA,Sv,%Sv,GAA,SO
35442,https://en.khl.ru/players/33314/,Casey Bailey,671,Regular season 2018/2019,246,20 Feb 2019,Jokerit - Slovan,7:1,25,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46321,https://en.khl.ru/players/29144/,David Boldizar,468,Regular season 2017/2018,246,20 Sep 2017,Slovan - Ak Bars,3:6,23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46322,https://en.khl.ru/players/29144/,David Boldizar,468,Regular season 2017/2018,246,23 Sep 2017,Vityaz - Slovan,4:0,23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46323,https://en.khl.ru/players/29144/,David Boldizar,468,Regular season 2017/2018,246,25 Sep 2017,CSKA - Slovan,3:2,23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46324,https://en.khl.ru/players/29144/,David Boldizar,468,Regular season 2017/2018,246,27 Sep 2017,Slovan - Vityaz,4:3,23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46325,https://en.khl.ru/players/29144/,David Boldizar,468,Regular season 2017/2018,246,3 Oct 2017,Slovan - Severstal,5:4 Б,23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46326,https://en.khl.ru/players/29144/,David Boldizar,468,Regular season 2017/2018,246,5 Oct 2017,Slovan - Torpedo,0:1,23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46327,https://en.khl.ru/players/29144/,David Boldizar,671,Regular season 2018/2019,246,22 Jan 2019,Dinamo R - Slovan,3:2,61,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46328,https://en.khl.ru/players/29144/,David Boldizar,671,Regular season 2018/2019,246,24 Jan 2019,Lokomotiv - Slovan,7:0,61,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46329,https://en.khl.ru/players/29144/,David Boldizar,671,Regular season 2018/2019,246,26 Jan 2019,Slovan - Dinamo Mn,2:4,61,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have multiple culprits here. Something must have went wrong with the way their data was stored. 

In [4]:
raw_players_match[raw_players_match['TOI'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 35442 to 354778
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   URL          23 non-null     object 
 1   Player name  23 non-null     object 
 2   IDSeason     23 non-null     int64  
 3   Season       23 non-null     object 
 4   Team         23 non-null     int64  
 5   Date         23 non-null     object 
 6   Teams        23 non-null     object 
 7   Score        23 non-null     object 
 8   №            23 non-null     int64  
 9   G            23 non-null     int64  
 10  Assists      23 non-null     int64  
 11  PTS          0 non-null      float64
 12  +/-          0 non-null      float64
 13  +            0 non-null      float64
 14  -            0 non-null      float64
 15  PIM          23 non-null     int64  
 16  ESG          0 non-null      float64
 17  PPG          0 non-null      float64
 18  SHG          0 non-null      float64
 19  OT

Most of the data is missing, and not because it is supposed to be a zero. After all, icetime cannot be zero if a player has participated in a match. And the only values present are integers, so definitely something weird with the formatting.

We do not know whether the player has zeroes in all those columns or if it is just a data storage issue. Since those are only a few broken rows, let us just drop them altogether. And, while we are at it, drop all rows where a player got no icetime.

In [5]:
# What values do we have in case of zero icetime?
raw_players_match.groupby('TOI').size()

TOI
-        9362
0       18734
0.0         1
0:01       16
0:02        8
        ...  
9:55      212
9:56      199
9:57      215
9:58      193
9:59      239
Length: 3694, dtype: int64

In [6]:
# No icetime can be recorded as either NaN, '-', '0' or '0.0' and we want neither of them.
zero_list = [np.NaN, '-', '0', '0.0']
players_match = raw_players_match[~raw_players_match['TOI'].isin(zero_list)].copy()

UPDATE: as it turns out, there is a number of rows for which the score is not actually a valid score. Let us look at it.

In [7]:
players_match[players_match['Score'] == '(-:-)']

,URL,Player name,IDSeason,Season,Team,Date,Teams,Score,№,G,...,BLS,FOA,W,L,SOP,GA,Sv,%Sv,GAA,SO
3335,https://en.khl.ru/players/14597/,Yegor Averin,167,Regular season 2009/2010,34,9 Jan 2010,Vityaz - Avangard,(-:-),29,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22895,https://en.khl.ru/players/14315/,Sergei Belokon,167,Regular season 2009/2010,19,9 Jan 2010,Vityaz - Avangard,(-:-),61,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24606,https://en.khl.ru/players/3431/,Viktor Bobrov,167,Regular season 2009/2010,19,9 Jan 2010,Vityaz - Avangard,(-:-),38,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29075,https://en.khl.ru/players/14653/,Vadim Berdnikov,167,Regular season 2009/2010,19,9 Jan 2010,Vityaz - Avangard,(-:-),36,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38095,https://en.khl.ru/players/15606/,Georgy Belousov,167,Regular season 2009/2010,19,9 Jan 2010,Vityaz - Avangard,(-:-),25,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42900,https://en.khl.ru/players/3950/,Anton Babchuk,167,Regular season 2009/2010,34,9 Jan 2010,Vityaz - Avangard,(-:-),78,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44249,https://en.khl.ru/players/10176/,Anton Belov,167,Regular season 2009/2010,34,9 Jan 2010,Vityaz - Avangard,(-:-),77,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45021,https://en.khl.ru/players/6428/,Alexei Bondarev,167,Regular season 2009/2010,34,9 Jan 2010,Vityaz - Avangard,(-:-),58,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52017,https://en.khl.ru/players/13252/,Rafael Batyrshin,167,Regular season 2009/2010,19,9 Jan 2010,Vityaz - Avangard,(-:-),23,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72353,https://en.khl.ru/players/15295/,Sergei Denisov,167,Regular season 2009/2010,19,9 Jan 2010,Vityaz - Avangard,(-:-),20,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,-,0.00,0.0


Can you see how all of those refer to a single match, Vityaz - Avangard on January 9th 2010?

That is a sort of unique match for KHL. As a result of two mass fights early into the game, both teams have racked up an enormous amount of penalties and did not have enough players to even continue playing the match. As such, it was cancelled and the score was recorded the way you see above, breaking the data processing workflow. Let us just drop those observations altogether.

In [8]:
players_match = players_match[players_match['Score'] != '(-:-)']

Now we can create a new column indicating whether a player is a skater or a goalie. Let us use the shifts for separation, since they are only tracked for skaters and goalies are supposed to have it as null.

In [9]:
players_match['Role'] = np.where(players_match['SFT'].isnull(), 'Goalie', 'Skater')

We have quite a bit of work ahead of us. Many columns contain data that we would like to see in other columns, such as years, home/visit team, whether the game was finished in the main time or in overtime/by shootouts and whether a player is a captain or assistant captain.

The icetime is currently stored in the 'minutes:seconds' format which cannot be used for analysis, so we need to create a separate column containing integer values of icetime in seconds. UPDATE: Some players turned out to already have their icetime recorded in seconds so we will need to fix that before converting.

In [10]:
def icetime_fix(icetime):
    time_list = icetime.split(':')
    
    if len(time_list) == 2:
        return icetime
    
    else:   
        minutes = 0
        seconds = int(time_list[0])
        while seconds >= 60:
            minutes += 1
            seconds -= 60
        return f'{minutes}:{seconds}'
        
def icetime_seconds(icetime):
    # To be used after turning all icetime values into a 'minutes:seconds' format.
    time_list = icetime.split(':')
    minutes = int(time_list[0])
    seconds = int(time_list[1])
    return minutes * 60 + seconds

In [11]:
# Separating the 'Season' column into the type of season and the years would allow us to more easily sort it.
players_match['Year'] = players_match['Season'].apply(lambda x: x[-9:])
players_match['Season'] = players_match['Season'].apply(lambda x: x[:-10])

# Now separating the 'Score' into each team's score, using a split by colon.
# In addition, we will create a 'Length' column that indicates in which period the game has ended.
# The split on a space separates the scores from an overtime indicator.
players_match['Home_score'] = players_match['Score'].apply(lambda x: x.split(' ')[0].split(':')[0])
players_match['Away_score'] = players_match['Score'].apply(lambda x: x.split(' ')[0].split(':')[1])

# We cannot just take the second element from the split since the list will only contain 1 element if there is no overtime.
# But we can artificially create an extra element in a list by padding the string with an extra whitespace at the end.
# This trick allows us to take the overtime indicator if it is present or a blank string if it is not.
players_match['Length'] = players_match['Score'].apply(lambda x: (x + ' ').split(' ')[1])

# We can use a similar approach to separate the 'Teams' column into each team's name and captaincy indicator.
# The splits used would be hyphen for team names and parenthesis for captaincy.
players_match['Home_team'] = players_match['Teams'].apply(lambda x: x.split('-')[0].split('(')[0].strip())
players_match['Away_team'] = players_match['Teams'].apply(lambda x: x.split('-')[1].split('(')[0].strip())

# Same as before, we need some padding so there is always at least 2 elements in the list.
# Also, the captaincy indicator is only 1 letter long and is not at the end of the string.
players_match['Captain'] = players_match['Teams'].apply(lambda x: (x + '( ').split('(')[1][0])

# Finally, let us fix the icetime and add icetime in seconds.
players_match['TOI'] = players_match['TOI'].apply(icetime_fix)
players_match['TOI_seconds'] = players_match['TOI'].apply(icetime_seconds)
players_match.head(5)

,URL,Player name,IDSeason,Season,Team,Date,Teams,Score,№,G,...,SO,Role,Year,Home_score,Away_score,Length,Home_team,Away_team,Captain,TOI_seconds
0,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season,54,28 Dec 2013,Barys - Amur,8:2,91,0,...,NaN,Skater,2013/2014,8,2,,Barys,Amur,,482
1,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season,54,3 Jan 2014,Amur - Lokomotiv,2:1,91,0,...,NaN,Skater,2013/2014,2,1,,Amur,Lokomotiv,,697
2,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season,54,5 Jan 2014,Amur - SKA,1:6,91,0,...,NaN,Skater,2013/2014,1,6,,Amur,SKA,,737
3,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season,54,7 Jan 2014,Amur - Atlant,2:3 Б,91,0,...,NaN,Skater,2013/2014,2,3,Б,Amur,Atlant,,105
4,https://en.khl.ru/players/16673/,Sergei Abramov,244,Regular season,54,9 Jan 2014,Amur - Severstal,1:3,91,0,...,NaN,Skater,2013/2014,1,3,,Amur,Severstal,,62


In [12]:
# What values do we have here?
players_match['Length'].unique()

array(['', 'Б', 'ОТ'], dtype=object)

The 'Score' column has contained a Russian letter 'Б'. It indicates shootouts and was apparently not properly changed to English. Therefore, we need to change it and, while we are at it, might as well change all values to the more obvious ones.

In [13]:
length_dict = {'': 'Standard', 'ОТ': 'Overtime', 'Б': 'Shootouts'}
players_match['Length'] = players_match['Length'].map(length_dict)

In [14]:
# What about here?
players_match['Captain'].unique()

array([' ', 'а', 'к', 'a', 'c', 'K', 'k', 'К', 'А', 'A'], dtype=object)

Same as with the scores, the letters used to indicate that the player is his team's captain ('К') or assistant captain ('А') are partially in Russian. Moreover, they seem to be using multiple different letters for the same thing, so let us fix it.

In [15]:
captain_dict = {' ': 'Player', 'а': 'Assistant', 'к': 'Captain', 'a': 'Assistant', 'c': 'Captain',
                'K': 'Captain','k': 'Captain', 'К': 'Captain', 'А': 'Assistant', 'A': 'Assistant'}
players_match['Captain'] = players_match['Captain'].map(captain_dict)

Currently, we can recognise a match by its date and at least one of the teams who participated in it. However, it would be more convenient to have match ID as a separate column rather than combining two columns every time.

In [16]:
players_match['Match_id'] = players_match.groupby(['Date', 'Home_team']).ngroup() + 1

We can now rearrange the columns.

In [17]:
# We can copy paste parts of the list of column names instead of typing them up manually.
players_match.columns

Index(['URL', 'Player name', 'IDSeason', 'Season', 'Team', 'Date', 'Teams',
       'Score', '№', 'G', 'Assists', 'PTS', '+/-', '+', '-', 'PIM', 'ESG',
       'PPG', 'SHG', 'OTG', 'GWG', 'SDS', 'SOG', '%SOG', 'FO', 'FOW', '%FO',
       'TOI', 'SFT', 'HITS', 'BLS', 'FOA', 'W', 'L', 'SOP', 'GA', 'Sv', '%Sv',
       'GAA', 'SO', 'Role', 'Year', 'Home_score', 'Away_score', 'Length',
       'Home_team', 'Away_team', 'Captain', 'TOI_seconds', 'Match_id'],
      dtype='object')

In [18]:
# Dropping the columns we are no longer interested in.
players_match.drop(['Teams', 'Score'], axis=1, inplace=True)

# We will have to move the columns around quite a bit.
columns = ['URL', 'Player name', 'Role', 'IDSeason', 'Season', 'Year', 'Team', 'Captain', 'Match_id', 'Date',
           'Home_team', 'Away_team', 'Home_score', 'Away_score', 'Length', '№', 'G', 'Assists', 'PTS', '+/-',
           '+', '-', 'PIM', 'ESG', 'PPG', 'SHG', 'OTG', 'GWG', 'SDS', 'SOG', '%SOG', 'FO', 'FOW', '%FO',
           'TOI', 'TOI_seconds', 'SFT', 'HITS', 'BLS', 'FOA', 'W', 'L', 'SOP', 'GA', 'Sv', '%Sv', 'GAA', 'SO']
players_match = players_match[columns]

# The current column names are not very informative, are they?
header = ['Profile', 'Player', 'Role', 'Season_id', 'Season', 'Year', 'Team_id', 'Captain', 'Match_id', 'Date', 'Home_team',
          'Away_team', 'Home_score', 'Away_score', 'Length', 'Number', 'Goals', 'Assists', 'Points', 'Plus_minus', 'Plus',
          'Minus', 'Penalties', 'Goals_even', 'Goals_powerplay', 'Goals_shorthanded', 'Goals_overtime', 'Game_winning_goals',
          'Game_winning_shootouts', 'Shots', 'Shots_percentage', 'Faceoffs', 'Faceoffs_won', 'Faceoffs_percentage',
          'Icetime', 'Icetime_seconds', 'Shifts', 'Hits', 'Shots_blocked', 'Penalties_against', 'Wins', 'Losses',
          'Shootouts', 'Goals_against', 'Saves', 'Saves_percentage', 'Goals_against_average', 'Shutouts']
players_match.columns = header

# The player's number and season/team/match IDs are currently stored as float, let us change them into object.
players_object = ['Season_id', 'Team_id', 'Match_id', 'Number']
players_match[players_object] = players_match[players_object].astype('object')

# What do you think we are going to do to the 'Date' column?
players_match['Date'] = pd.to_datetime(players_match['Date'])
players_match.head(5)

,Profile,Player,Role,Season_id,Season,Year,Team_id,Captain,Match_id,Date,...,Shots_blocked,Penalties_against,Wins,Losses,Shootouts,Goals_against,Saves,Saves_percentage,Goals_against_average,Shutouts
0,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,6994,2013-12-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,7870,2014-01-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,9038,2014-01-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,9702,2014-01-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,10344,2014-01-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Can we start changing the data types? Not really.

Most of our columns still has many NaN values because different statistics are tracked for skaters and goalies. And integers do not like having NaN values in them. It could be worked around but such an approach would not necessarily be the best one.

We could, of course, leave it as it is or replace missing values with zeros. However, analysing skaters and goalies together in the future sounds like a bad analysis design since the two groups are very distinct. Therefore, let us separate the data into two distinct dataframes and store skater statistics and goalie statistics separately. That way, we can also change floats into integers within each dataframe separately.

In [19]:
# Thankfully, we have a convenient column to separate on.
skaters_match = players_match[players_match['Role'] == 'Skater']
goalies_match = players_match[players_match['Role'] == 'Goalie']

In [20]:
# Most columns are either null or non-null in every row except for 'Hits', 'Shots_blocked' and 'Penalties_against'.
skaters_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399663 entries, 0 to 451104
Data columns (total 48 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Profile                 399663 non-null  object        
 1   Player                  399663 non-null  object        
 2   Role                    399663 non-null  object        
 3   Season_id               399663 non-null  object        
 4   Season                  399663 non-null  object        
 5   Year                    399663 non-null  object        
 6   Team_id                 399663 non-null  object        
 7   Captain                 399663 non-null  object        
 8   Match_id                399663 non-null  object        
 9   Date                    399663 non-null  datetime64[ns]
 10  Home_team               399663 non-null  object        
 11  Away_team               399663 non-null  object        
 12  Home_score              399663

In [21]:
# And all is perfect here!
goalies_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23283 entries, 35 to 450609
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Profile                 23283 non-null  object        
 1   Player                  23283 non-null  object        
 2   Role                    23283 non-null  object        
 3   Season_id               23283 non-null  object        
 4   Season                  23283 non-null  object        
 5   Year                    23283 non-null  object        
 6   Team_id                 23283 non-null  object        
 7   Captain                 23283 non-null  object        
 8   Match_id                23283 non-null  object        
 9   Date                    23283 non-null  datetime64[ns]
 10  Home_team               23283 non-null  object        
 11  Away_team               23283 non-null  object        
 12  Home_score              23283 non-null  obje

What is going on here? It feels as if for some reason those three columns are only recorded part of the time. Is it happening all the time?

In [22]:
# Distribution of NaN values for 'Hits' by season.
skaters_match[skaters_match['Hits'].isnull()].groupby(['Season', 'Year']).size()

Season          Year     
Playoffs        2008/2009     2241
                2009/2010     2659
                2010/2011     3174
                2011/2012     3103
                2012/2013     3161
                2013/2014     3176
Regular season  2008/2009    24804
                2009/2010    25299
                2010/2011    23044
                2011/2012    23167
                2012/2013    25681
                2013/2014    28986
dtype: int64

Would you look at that, the missing values are only the case for seasons 2008/2009 through 2013/2014! Clearly, the indicators have just not been tracked in though years. Mystery solved.

However, what should we do with it? Replacing it with zeros would not be very fair and can mess up our analysis. At the same time, we cannot change the column to integers without replacing the NaN values. Oh well, we might leave it as is for now and keep in mind that any further analysis needs to take into account that change from season 2014/2015 onwards.

We are going to remove the null columns and change the non-null ones to integers. Actually, in a few cases we would need to change the columns to floats for things such as '%SOG' (percentage of shots on goal that scored) which seem to be stored as objects right now.

At the same time, Some columns that we want to store as floats have '-' for their value, which cannot be converted into a float value. This is because they are obtained by dividing one statistics by another and one of the two may not be suitable for such operation. We are going to replace those values with NaN.

In [23]:
# We can copy paste parts of the previously created list of column names instead of typing them up manually.
print(header)

['Profile', 'Player', 'Role', 'Season_id', 'Season', 'Year', 'Team_id', 'Captain', 'Match_id', 'Date', 'Home_team', 'Away_team', 'Home_score', 'Away_score', 'Length', 'Number', 'Goals', 'Assists', 'Points', 'Plus_minus', 'Plus', 'Minus', 'Penalties', 'Goals_even', 'Goals_powerplay', 'Goals_shorthanded', 'Goals_overtime', 'Game_winning_goals', 'Game_winning_shootouts', 'Shots', 'Shots_percentage', 'Faceoffs', 'Faceoffs_won', 'Faceoffs_percentage', 'Icetime', 'Icetime_seconds', 'Shifts', 'Hits', 'Shots_blocked', 'Penalties_against', 'Wins', 'Losses', 'Shootouts', 'Goals_against', 'Saves', 'Saves_percentage', 'Goals_against_average', 'Shutouts']


In [24]:
skaters_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399663 entries, 0 to 451104
Data columns (total 48 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Profile                 399663 non-null  object        
 1   Player                  399663 non-null  object        
 2   Role                    399663 non-null  object        
 3   Season_id               399663 non-null  object        
 4   Season                  399663 non-null  object        
 5   Year                    399663 non-null  object        
 6   Team_id                 399663 non-null  object        
 7   Captain                 399663 non-null  object        
 8   Match_id                399663 non-null  object        
 9   Date                    399663 non-null  datetime64[ns]
 10  Home_team               399663 non-null  object        
 11  Away_team               399663 non-null  object        
 12  Home_score              399663

In [25]:
# Starting with the more numerous skaters.
skaters_match.drop(['Wins', 'Losses', 'Shootouts', 'Goals_against', 'Saves', 'Saves_percentage',
                     'Goals_against_average', 'Shutouts'], axis=1, inplace=True)

# Columns to be changed into integers.
skaters_int = ['Home_score', 'Away_score', 'Goals', 'Assists', 'Points', 'Plus_minus', 'Plus', 'Minus',
               'Penalties', 'Goals_even', 'Goals_powerplay', 'Goals_shorthanded', 'Goals_overtime',
               'Game_winning_goals', 'Game_winning_shootouts', 'Shots', 'Faceoffs', 'Faceoffs_won']
skaters_match[skaters_int] = skaters_match[skaters_int].astype('int')

# We cannot directly convert shifts which are in float format but stored as objects.
skaters_match['Shifts'] = skaters_match['Shifts'].astype('float').astype('int')

# Columns to be changed into floats.
skaters_float = ['Shots_percentage', 'Faceoffs_percentage']
skaters_match[skaters_float] = skaters_match[skaters_float].replace('-', np.NaN).astype('float')
skaters_match.head(5)

C:\Work\Software\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Work\Software\Anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-25-f3c913f3f1f8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

,Profile,Player,Role,Season_id,Season,Year,Team_id,Captain,Match_id,Date,...,Shots_percentage,Faceoffs,Faceoffs_won,Faceoffs_percentage,Icetime,Icetime_seconds,Shifts,Hits,Shots_blocked,Penalties_against
0,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,6994,2013-12-28,...,0.0,0,0,NaN,8:02,482,12,NaN,NaN,NaN
1,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,7870,2014-01-03,...,0.0,1,0,0.0,11:37,697,14,NaN,NaN,NaN
2,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,9038,2014-01-05,...,0.0,0,0,NaN,12:17,737,13,NaN,NaN,NaN
3,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,9702,2014-01-07,...,NaN,0,0,NaN,1:45,105,2,NaN,NaN,NaN
4,https://en.khl.ru/players/16673/,Sergei Abramov,Skater,244,Regular season,2013/2014,54,Player,10344,2014-01-09,...,NaN,0,0,NaN,1:02,62,1,NaN,NaN,NaN


In [26]:
# Now for the goalies.
goalies_match.drop(['Points', 'Plus_minus', 'Plus', 'Minus', 'Goals_even', 'Goals_powerplay', 'Goals_shorthanded',
                     'Goals_overtime', 'Game_winning_goals', 'Game_winning_shootouts', 'Shots_percentage',
                     'Faceoffs', 'Faceoffs_won', 'Faceoffs_percentage', 'Shifts', 'Hits', 'Shots_blocked', 'Penalties_against'],
                    axis=1, inplace=True)

# Columns to be changed into integers.
goalies_int = ['Goals', 'Assists', 'Penalties', 'Shots', 'Wins', 'Losses', 'Shootouts', 'Goals_against',
               'Saves', 'Shutouts']
goalies_match[goalies_int] = goalies_match[goalies_int].astype('int')

# Columns to be changed into floats.
goalies_float = ['Saves_percentage', 'Goals_against_average']
goalies_match[goalies_float] = goalies_match[goalies_float].replace('-', np.NaN).astype('float')
goalies_match.head(5)

C:\Work\Software\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Work\Software\Anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Profile,Player,Role,Season_id,Season,Year,Team_id,Captain,Match_id,Date,...,Icetime,Icetime_seconds,Wins,Losses,Shootouts,Goals_against,Saves,Saves_percentage,Goals_against_average,Shutouts
35,https://en.khl.ru/players/16462/,Maxim Alyapkin,Goalie,266,Regular season,2014/2015,26,Player,1413,2015-01-13,...,34:16,2056,0,1,0,2,9,81.8,3.50,0
45,https://en.khl.ru/players/16462/,Maxim Alyapkin,Goalie,309,Regular season,2015/2016,26,Player,4698,2015-12-22,...,32:06,1926,1,0,0,1,6,85.7,1.87,0
46,https://en.khl.ru/players/16462/,Maxim Alyapkin,Goalie,309,Regular season,2015/2016,26,Player,6206,2015-12-26,...,28:19,1699,0,1,0,2,4,66.7,4.24,0
332,https://en.khl.ru/players/16898/,Artyom Artemyev,Goalie,244,Regular season,2013/2014,56,Player,10179,2013-09-08,...,20:00,1200,0,0,0,3,13,81.2,9.00,0
338,https://en.khl.ru/players/16898/,Artyom Artemyev,Goalie,244,Regular season,2013/2014,56,Player,4997,2013-09-22,...,0:15,15,0,0,0,0,0,NaN,0.00,0


Everything seems to be in order, good job us! Now for the best part.

In [27]:
skaters_match.to_csv('../data/skaters_match.csv', encoding='utf8', index=False)
goalies_match.to_csv('../data/goalies_match.csv', encoding='utf8', index=False)